## Steps to be followed for the project

- Importing necessary Libraries
- Creating S3 bucket
- Mapping train And Test Data in S3
- Mapping The path of the models in S3

In [31]:
import sagemaker
import boto3 # with help of python from out local environment we can read the "public" S3 bucket

from sagemaker.amazon.amazon_estimator import get_image_uri
# we will load an image container which has XGBOOST algorithms and can be downloaded from get_image_uri 

from sagemaker.session import s3_input, Session

In [32]:
bucket_name = 'bankapplication05312022'

my_region= boto3.session.Session().region_name

print(my_region)

#to access the S3 bucket we need to create a session as well

us-east-1


In [33]:
s3= boto3.resource('s3') #that's how we get the access of S3 bucket


try:
    if my_region == "us-east-1":
        s3.create_bucket(Bucket=bucket_name,CreateBucketConfiguration={'LocationConstraint': 'us-east-1'})
    print(' Bucket is created')

except Exception as e:
    print('S3 error : ', e)

S3 error :  An error occurred (InvalidLocationConstraint) when calling the CreateBucket operation: The specified location-constraint is not valid


In [34]:
# set an output path where the trained model will be saved

prefix = 'xgboost-as-a-built-in-algo'
output_path ='s3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://bankapplication05312022/xgboost-as-a-built-in-algo/output


In [35]:
import pandas as pd
import urllib

#renaming the csv file to bank_clean

try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)
    
    
# data is downloaded into the instance i.e. our jupyter notebook ( which is running on an instance)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [36]:
### Train Test split

import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


In [48]:
# Whenever we are working in sagemaker "Dependent variable should be our 1st feature in dataframe
### Saving Train And Test Into Buckets
## We start with Train Data
import os
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)

#accessing the s3 bucket and uploading
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')

#now we have access the s3 bucket to get the path data or Data
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [49]:
s3_input_train

In [ ]:
## We save with test Data

In [50]:
import os
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)

#accessing the s3 bucket and uploading
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')

#now we have access the s3 bucket to get the path data or Data
s3_input_test = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

In [ ]:
# algorithms are present in containers or as images. we hae to pull them into our instances 

In [51]:
# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
container = get_image_uri(boto3.Session().region_name,'xgboost', repo_version='1.0-1')

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [ ]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }


#got this hyperparameters by EDA on local system as it would cost much on cloud

In [ ]:

# SageMaker estimator that calls the xgboost-container

estimator = sagemaker.estimator.Estimator(image_name=container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          train_instance_count=1, 
                                          train_instance_type='ml.m5.2xlarge', 
                                          train_volume_size=5, # 5 GB 
                                          output_path=output_path,
                                          train_use_spot_instances=True,
                                          train_max_run=300,
                                          train_max_wait=600)

In [ ]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

## Deploy Machine Learning Model As Endpoints

In [ ]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')


In [ ]:
from sagemaker.predictor import csv_serializer
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
xgb_predictor.content_type = 'text/csv' # set the data type for an inference
xgb_predictor.serializer = csv_serializer # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

## Confusion matrix 

In [ ]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))

## Deleting The Endpoints

In [ ]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()